# Dependencies

## Install Libraries

In [ ]:
!pip install geemap
!pip install geopandas
# !pip install imgix
# !pip install boto3
!pip install eefolium
!pip install geojson

## Import Libraries

In [ ]:
import google.colab
import ee
import eefolium
import geemap
import geopandas as gpd
import pandas as pd
import altair as alt
import numpy as np
import folium
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.dates as dates
import seaborn as sns
from PIL import Image
from datetime import date
from datetime import datetime
from datetime import datetime as dt
import datetime
import geojson
from IPython.display import Image, HTML

## Load and Authenticate Earth Engine (Manual Touchpoint)

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# AOI

## Define Area of interest from geojson

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
with open('/content/drive/MyDrive/Farmlands_ifarmer/Fields_geojson/Rice_farmland.geojson') as f:
    gj_maize = geojson.load(f)

df_geo_maize = pd.DataFrame.from_dict(gj_maize)

df_geo_maize['polygon'] = df_geo_maize.features.apply(lambda x: x["geometry"]["coordinates"])

df_geo_maize['crop'] = df_geo_maize.features.apply(lambda x: x["properties"]["crop"])
df_geo_maize['farmer'] = df_geo_maize.features.apply(lambda x: x["properties"]["name"])

a_farm = df_geo_maize['features'][0]['geometry']['coordinates']
df_geo_maize['farmer']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_geo_maize['features'][0]['geometry']

In [ ]:
df_geo_maize['features'][0]['properties']

In [ ]:
aoi = ee.Geometry.Polygon(a_farm)


## Define AOI from FC-GEE

In [ ]:
field_id = "ACMM_br01_1001"
aoi = ee.FeatureCollection("projects/ee-ifarmer/assets/br01/br01_farmlands").filter(
    ee.Filter.eq('field_id', field_id))

## AOI from lat long

In [ ]:
# aoi = ee.FeatureCollection( # Jalal uddin m
#         [ee.Feature(
#             ee.Geometry.Polygon(
#                 [[[89.35090823579667, 25.98489032294839],
#           [89.35084386278031, 25.984721548383302],
#           [89.35111744809983, 25.984639572078546],
#           [89.35112281251786, 25.98432613273902],
#           [89.35115499902604, 25.983998226074],
#           [89.35146077085373, 25.983901782763148],
#           [89.35154660154221, 25.983978937418154],
#           [89.35161097455857, 25.984210401079338],
#           [89.35151977945206, 25.984683113140704],
#           [89.35128910947678, 25.98477473367891]]]))])

## Map the AOI

In [ ]:
# # Generate map with base map
# Map = geemap.Map()
# Map.add_basemap('HYBRID')

# # Original Farmland
# Map.addLayer(aoi, {}, 'Aoi')
# # Center the area of Interest
# Map.centerObject(aoi, zoom = 20)
# Map

# Image preprocessing

## Function to Calculate and Add New Bands

In [ ]:
# Add NDRE/NDVI/NDMI/MSAVI Bands

# This field contains UNIX time in milliseconds.


def addVariables(image):
  # Compute time in fractional years since the epoch.
  # Get time from metadata of image (converted from milisecond)
  timeField = 'system:time_start'
  date = ee.Date(image.get(timeField))
  # Start conting the year from 1970 to present
  years = date.difference(ee.Date('1970-01-01'), 'year')
  # Return the image with the added bands.
  image = (image
          # Add an NDVI band.
          .addBands(image.normalizedDifference(['B8', 'B4']).rename('ndvi')).float()
          # Add an NDMI band
          .addBands(image.normalizedDifference(['B8', 'B11']).rename('ndmi')).float()
          # Add an MSAVI band
          .addBands(image.expression(
          '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2',
          {
            'NIR': image.select('B8'),
            'RED': image.select('B4')
          }).rename('msavi')).float()
          # Add an NDRE band
          .addBands(image.normalizedDifference(['B8', 'B5']).rename('ndre')).float()
          # Add a time band.
          .addBands(ee.Image(years).rename('t').float())
          # Add a constant band.
          .addBands(ee.Image.constant(1)))

  # return image with additional bands
  return image

## Cloud masking

### Cloud Mask Parameters

In [ ]:
start_year = '2019'
end_year = '2024'
CLOUD_FILTER = 95
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

In [ ]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [ ]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [ ]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [ ]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

# Query Sentinel 2 Image Collections (Both Level 1C and Level 2A)

In [ ]:
# Query two different qualities of Sentinel 2 collections
# Add Bands to every satellite image in each Sentinel 2 collection
# Merge the two Sentinel 2 collections together
# .filterMetadata("CLOUDY_PIXEL_OVER_LAND_PERCENTAGE", "less_than", 50)

# Add bands to Satellite Image to Sentinel 2 (Level 2-A)
# filteredSentinelCollection1 = (ee.ImageCollection("COPERNICUS/S2_SR")
#                               .filterBounds(aoi)
#                               .filterDate('2018', '2024')
#                               .map(addVariables))
                              # .map(add_cld_shdw_mask)
                              # .map(apply_cld_shdw_mask));

# # Add bands to Satellite Image to Sentinel 2 (Level 1-C)
# filteredSentinelCollection2 = ee.ImageCollection("COPERNICUS/S2") \
#                               .filterBounds(aoi) \
#                               .filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 90) \
#                               .filterDate('2018', '2024') \
#                               .map(addVariables);
def get_s2_sr_cld_col(aoi, start_year, end_year):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_year, end_year)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_year, end_year))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

# Merge the two Sentinel 2 Image Collections

In [ ]:
# Merge two imagecollection together
#filteredSentinelCollection = merge_two_image_collections(filteredSentinelCollection1, filteredSentinelCollection2)

# filteredSentinelCollection = filteredSentinelCollection1 \
#                             .merge(filteredSentinelCollection2) \
#                             .sort('system:time_start', True);

# filteredSentinelCollection = filteredSentinelCollection1
s2_sr_cld_col = get_s2_sr_cld_col(aoi, start_year, end_year)
filteredSentinelCollection = s2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).map(addVariables)
# filteredSentinelCollection

# Define Scale and Projection

In [ ]:
sentinel_pro = filteredSentinelCollection.first().select('ndvi').projection();
CopyScale = sentinel_pro.nominalScale();
# CopyScale
# print(CopyScale, 'original scale Sentinel (m)')

In [ ]:
def sentinel_resample(image):
  return image.reproject(sentinel_pro, None, 10).reduceResolution(
      **{
      "reducer": ee.Reducer.mean(),
      "maxPixels": 1024
  }).copyProperties(image)
#insert here the desired scale in meters
#Force the next reprojection to aggregate instead of resampling.

In [ ]:
sentinelResample = filteredSentinelCollection.map(sentinel_resample)

# Functions

## Spaced Points function

In [ ]:
def spacedPoints(AOI, projection):
  # make a coordinate image
  # get coordinates image
  latlon = ee.Image.pixelLonLat().reproject(projection);
  # put each lon lat in a list -> this time for getting an multipoint list (more useful inside the GEE)
  coords = (latlon.select(['longitude', 'latitude']).reduceRegion(
      **{'reducer': ee.Reducer.toList(),
         'geometry': AOI,
         'scale': projection.nominalScale().toInt(),
         'maxPixels': 10e10
         }));
  # zip the coordinates for representation. Example: zip([1, 3],[2, 4]) --> [[1, 2], [3,4]]
  point_list = ee.List(coords.get('longitude')).zip(ee.List(coords.get('latitude')));
  # preset a random list
  listp = ee.List([0]);
  # map over the point list and add features as geometry
  def point_func(point):
    ind = point_list.indexOf(point);
    feat = ee.Feature(ee.Geometry.Point(point_list.get(ind)), {'ID': ind});
    return listp.add(feat);

  feats = ee.FeatureCollection(point_list.map(point_func).flatten().removeAll([0]));
  return feats;

## Zonal Statistics function

In [ ]:
def zonalStats(ic, fc, params):
  # Initialize internal params dictionary.
  _params = {
      'reducer': ee.Reducer.mean(),
      'scale': None,
      'crs': None,
      'bands': None,
      'bandsRename': None,
      'imgProps': None,
      'imgPropsRename': None,
      'datetimeName': 'datetime',
      'datetimeFormat': 'YYYY-MM-dd'
      };

  # Replace initialized params with provided params.
  if params:
    for param in params:
      _params[param] = params[param] or _params[param];

  # Set default parameters based on an image representative.
  imgRep = ic.first();
  nonSystemImgProps = ee.Feature(None).copyProperties(imgRep).propertyNames()
  if not _params['bands']:
    _params['bands'] = imgRep.bandNames()
  if not _params['bandsRename']:
    _params['bandsRename'] = _params['bands'];
  if not _params['imgProps']:
    _params['imgProps'] = nonSystemImgProps;
  if not _params['imgPropsRename']:
    _params['imgPropsRename'] = _params['imgProps'];

  def img_func(img):
    # Select bands (optionally rename), set a datetime & timestamp property.
    img = ee.Image(img.select(_params['bands'], _params['bandsRename'])) \
    .set(_params['datetimeName'], img.date().format(_params['datetimeFormat'])) \
    .set('timestamp', img.get('system:time_start'));

    # Define final image property dictionary to set in output features.
    propsFrom = ee.List(_params['imgProps']) \
    .cat(ee.List([_params['datetimeName'], 'timestamp']));
    propsTo = ee.List(_params['imgPropsRename']) \
    .cat(ee.List([_params['datetimeName'], 'timestamp']));
    imgProps = img.toDictionary(propsFrom).rename(propsFrom, propsTo);

    # Subset points that intersect the given image.
    fcSub = fc.filterBounds(img.geometry());

    def f_func(f):  # Add Metadata to each feature
      return f.set(imgProps)

    # Reduce the image by regions.
    return img.reduceRegions(
        **{
        'collection': fcSub,
        'reducer': _params['reducer'],
        'scale': _params['scale'],
        'crs': _params['crs']
        }).map(f_func);

  # Map the reduceRegions function over the image collection.
  results = ic.map(img_func).flatten().filter(ee.Filter.notNull(_params['bandsRename']));

  return results;

# Define Date Range

In [ ]:
dates = pd.date_range(start='01/17/2022', end='06/12/2022', freq='5D') #month/date/year
t_date = "2022/01/17"

reference_date = np.datetime64('2022-01-17')

## Merged GDF

In [ ]:
merged_gdf = pd.DataFrame() # Empty df

time_start_overall = datetime.datetime.now()
flag = False

for i in range(len(dates)-1):
  start_date = dates[i]
  end_date = dates[i+1]

  latestImg = sentinelResample.filterDate(start_date, end_date).first().clip(aoi)

  # NDVI
  ndvi = latestImg.select('ndvi');
  # NDMI
  ndmi = latestImg.select('ndmi');
  # MSAVI
  msavi = latestImg.select('msavi');
  # NDRE
  ndre = latestImg.select('ndre');

  projection = ndvi.projection();

  pts = spacedPoints(aoi, projection);

  ptsTopo = pts;

  # Concatenate four Bands of the Image
  vi = ee.Image.cat(ndvi,ndmi, msavi, ndre)

  # Wrap the single image in an ImageCollection for use in the zonalStats function.
  viCol = ee.ImageCollection([vi]);

  # Define parameters for the zonalStats function.
  params = {
      'bands': [0, 1, 2, 3],
      'bandsRename': ['ndvi','ndmi', 'msavi', 'ndre']}

  # Extract zonal statistics per point per image.
  ptsTopoStats = zonalStats(viCol, ptsTopo, params)
  merged_data = ptsTopoStats

  # print(merged_data.size())
  # if ee.Number(merged_data.size()) > 0:
  #   pass
  # else:
  #   print(start_date)
  #   continue

  # For loop for Featurecollection to geoDataframe
  count = 1
  chunk = 5000
  selectors = ['ndvi', 'ndre', 'msavi', 'ndmi', 'timestamp', 'CLOUDY_PIXEL_PERCENTAGE']

  start_time = datetime.datetime.now()
  for i in range(0,10000000, chunk):
    time_start = datetime.datetime.now()

    subset_data = ee.FeatureCollection(merged_data.toList(chunk, i))
    # print(datetime.datetime.now() - time_start, count, 'To list')

    start_time_gdf = datetime.datetime.now()

    try:
      subset_gdf = geemap.ee_to_geopandas(subset_data.select(selectors))
    except:
      flag = True
      print(start_date)
      break
    # print(datetime.datetime.now() - start_time_gdf, count, 'To gdf')
    merged_gdf = pd.concat([merged_gdf, subset_gdf], ignore_index=True)
    # merged_gdf = merged_gdf.append(subset_gdf, ignore_index=True)

    # print(datetime.datetime.now() - time_start, count)

    if len(subset_gdf) < chunk:
      # print(len(subset_gdf), i)
      duration = datetime.datetime.now() - start_time
      # print(duration, 'finish')
      break

    count += 1

  if flag == True:
    continue

print(datetime.datetime.now() - time_start_overall)

In [ ]:
# (merged_data).type()

In [ ]:
merged_gdf.head(3)
# merged_gdf.to_csv("merged_gdf.csv", index = False)

In [ ]:
merged_gdf['timestamp'] = pd.to_datetime(merged_gdf.timestamp,unit='ms',)
sat_df = merged_gdf[:] #[merged_gdf.timestamp > datetime.datetime(2022,9,1)]

In [ ]:
sat_df.head(3)
# sat_df.to_csv('sat_df_cloud_2020.csv',index=False)

In [ ]:
# sat_df[sat_df.CLOUDY_PIXEL_OVER_LAND_PERCENTAGE>70]

## GDF Function Depricated

In [ ]:
# def get_pixel_df(df):

#     list_cols = list(df.timestamp.unique())
#     print(list_cols)
#     geo_id = list(df.geo.unique())


#     sub_df = pd.DataFrame(columns=list_cols,index=geo_id)

#     i=0
#     for id_c in geo_id:



#         df_1 = df[df.geo == id_c]


#         for dis in list_cols:


#             sub_df[dis][i] = df_1[df_1.timestamp==dis].ndvi.values[0]

#         i+=1

#     return sub_df.reset_index().rename(columns={'index':'geo_id'})

## GDF Function

In [ ]:
def get_efficient_pixelDF(df):
    sub_df = df.pivot(index='geo', columns='timestamp', values='ndre')
    return sub_df

In [ ]:
ndre_df = sat_df[['ndre','timestamp','geometry']]
# ndre_df['timestamp']=pd.to_datetime(ndre_df.timestamp,unit='ms',)

# ndre_df.rename(columns={'geometry':'geo'},inplace=True)
ndre_df['geo'] = ndre_df['geometry'].apply(lambda x: str(x))

ndre_df['geo']=ndre_df['geo'].str[7:-1]
ndre_df[["long", "lat"]] = ndre_df["geo"].str.split(" ", expand=True)

ndre_df['long'] = ndre_df['long'].str[0:13]
ndre_df['lat'] = ndre_df['lat'].str[0:13]
ndre_df['geo'] = ndre_df.long+','+ndre_df.lat

ndre_df['timestamp'] = ndre_df['timestamp'].dt.date
# ndre_df['timestamp'] = ndre_df['timestamp'].dt.date
# ndre_df.sort_values('timestamp', inplace=True)
ndre_df.timestamp = ndre_df.timestamp.astype('str')

In [ ]:
%%time
df = get_efficient_pixelDF(ndre_df)
df.columns.name = None
df_main = get_efficient_pixelDF(ndre_df)
df_main.columns.name = None

In [ ]:
df.head()

In [ ]:
# df.head(100)
df= df.reset_index().rename(columns={'geo':'geo_id'})
# df.to_excel('gdf_data.xlsx',index=False)

In [ ]:
# df
# df_main.to_excel('main_df.xlsx',index=False)

## Matrix DF

In [ ]:
df_short = df[:]

In [ ]:
df_copy = df[:]
# df_copy = df.copy()
# df_copy
# df.to_csv("df_matrix.csv", index = False)

In [ ]:
def age_range(data_frame):
    filtered_columns_4 = [col for col in data_frame.columns.values if ((isinstance(col, int) and 30 <= col < 115) or (isinstance(col, str) and col == 'geo_id'))]

    filtered_columns_1 = [col for col in data_frame.columns.values if ((isinstance(col, int) and 40 <= col < 65) or (isinstance(col, str) and col == 'geo_id'))]
    filtered_columns_2 = [col for col in data_frame.columns.values if ((isinstance(col, int) and 65 <= col < 95) or (isinstance(col, str) and col == 'geo_id'))]
    filtered_columns_3 = [col for col in data_frame.columns.values if ((isinstance(col, int) and 95 <= col < 115) or (isinstance(col, str) and col == 'geo_id'))]


    data_frame.columns = data_frame.columns.astype(str)

    filtered_df_1 = data_frame[[str(element) for element in filtered_columns_1]]
    filtered_df_2 = data_frame[[str(element) for element in filtered_columns_2]]
    filtered_df_3 = data_frame[[str(element) for element in filtered_columns_3]]
    filtered_df_4 = data_frame[[str(element) for element in filtered_columns_4]]

    return filtered_df_1, filtered_df_2, filtered_df_3, filtered_df_4

In [ ]:
def df_2_day(df_m, t_date):
    data_frame = df_m.copy()
    col_name = list(data_frame.columns.values)
    for i in range(len(col_name)-1):
        new_date = (lambda x: x.replace("-", "/"))(str(col_name[i+1]))
        t_date = (lambda x: x.replace("-", "/"))(t_date)

        # difference between dates in timedelta
        res = (dt.strptime(new_date, "%Y/%m/%d") - dt.strptime(t_date, "%Y/%m/%d")).days
        data_frame.columns.values[i+1] = res
    # return df

    df1, df2, df3, df4 = age_range(data_frame)
    return df1, df2, df3, df4

In [ ]:
df_4065, df_6595, df_95115, df_30115 = df_2_day(df_copy, t_date)

In [ ]:
# df_30115.to_csv('df_30115_2020_tree.csv', index = False)
df_30115.isna().sum()

In [ ]:
df_30115.head(100)

In [ ]:
df.shape

## Matrix Define

In [ ]:
matrix_cols_4065 = ['<0', '0-.01', '.01-.06', '.06-.08', '.08-.10', '.10-.12', '.12-.14', '.14-.17', '>.17']
list_col_range_4065 = [-1, 0, .01, .06, .08, .1, .12, .14, .17, 1]

matrix_cols_6595 = ['<0', '0-.1', '.1-.15', '.15-.2', '.2-.27', '.27-.32', '.32-.37', '.37-.42', '>.42']
list_col_range_6595 = [-1, 0, .1, .15, .2, .27, .32, .37, .42, 1]

matrix_cols_95115 = ['<0', '0-.01', '.01-.06', '.06-.09', '.09-.10', '.10-.12', '.12-.17', '.17-.22', '>.22']
list_col_range_95115 = [-1, 0, .01, .06, .09, .10, .12, .17, .22, 1]
# crop_mapper = {'paddy': {'stage_1': {'weight': .2, ndvi_ranges: [-1, 0, .2, .3, .35, .4, .45, .5, .55, 1]}}}


matrix_cols = ['<0', '0-.1', '.1-.2', '.2-.3', '.3-.35', '.35-.4', '.4-.5', '.5-.7', '>.7']
# matrix_cols = ['<0', '0-.1', '.1-.2', '.2-.3','>.3']
list_col_range = [-1, 0, .1, .2, .3, .35, .4, .5, .7, 1]
# list_col_range = [-1, 0, .1, .2, .3, 1]


# matrix_rows = ['<10%', '10-30%', '30-50%', '50-70%', '70-90%', '>90%']
# list_row_range = [0, .1, .3, .5, .7, .9, 1]

# df_matrix = pd.DataFrame(0, index=matrix_rows, columns=matrix_cols)
# # df_matrix

## Matrix

In [ ]:
def df_2_mat(dataframe, mat_col, list_col_ran):

    mat_row = ['<10%', '10-30%', '30-50%', '50-70%', '70-90%', '>90%']
    list_row_ran = [0, .1, .3, .5, .7, .9, 1]

    mat = pd.DataFrame(0, index=mat_row, columns=mat_col)

    pixels = list(dataframe.geo_id.unique())
    count_val = dataframe.shape[1] - 1
    count = 0
    for pixel in pixels:
        ndvi_count_dict = dict.fromkeys(mat_col, 0)
        ndvi_vals = list(dataframe.iloc[count].values)[1:]
        for ndvi in ndvi_vals:
            for i in range(len(list_col_ran)-1):
                start_val = list_col_ran[i]
                end_val = list_col_ran[i+1]
                if ndvi>=start_val and ndvi<=end_val:
                    ndvi_count_dict[mat_col[i]] += 1
                    break

        ndvi_ratio_dict = {key: ndvi_count_dict[key]/count_val for key, val in ndvi_count_dict.items() if val>0}
    #     print(ndvi_ratio_dict)

        for k,v in ndvi_ratio_dict.items():
            for j in range(len(list_row_ran)-1):
                start_val = list_row_ran[j]
                end_val = list_row_ran[j+1]
                if v>=start_val and v<=end_val:
                    mat.at[mat_row[j], k] = mat.at[mat_row[j], k] + 1
                    break

        count += 1
    return mat

In [ ]:
df_matrix_4065 = df_2_mat(df_4065, matrix_cols_4065, list_col_range_4065)
df_matrix_6595 = df_2_mat(df_6595, matrix_cols_6595, list_col_range_6595)
df_matrix_95115 = df_2_mat(df_95115, matrix_cols_95115, list_col_range_95115)

In [ ]:
df_matrix_4065

In [ ]:
df_matrix_6595

In [ ]:
df_matrix_95115

In [ ]:
# df_matrix
# /len(df_short)*100

In [ ]:
def mat_2_fscore(mat, length):
    length = int(length)
    row,col = mat.shape
    farm_score_sum = 0
    max_score = 10
    count_ = 0
    for i in range(row, row-4, -1):
      for j in range(col, col-3, -1):

        pixel_score = max(mat.iloc[i-1,j-1]*(max_score - ((row-i) + (col-j+(count_)))), 0)
        farm_score_sum += pixel_score
        # print(i, j, pixel_score)

        count_ += 1

      farm_score_mat = farm_score_sum/(length * max_score)*10

    return farm_score_mat

In [ ]:
farm_score_4065 = mat_2_fscore(df_matrix_4065, len(df_4065))
farm_score_4065

In [ ]:
farm_score_6595 = mat_2_fscore(df_matrix_6595, len(df_6595))
farm_score_6595

In [ ]:
farm_score_95115 = mat_2_fscore(df_matrix_95115, len(df_95115))
farm_score_95115

## Farm Score Matrix

In [ ]:
farm_score_mat = (farm_score_4065*.15 + farm_score_6595*.65 + farm_score_95115*.2)
farm_score_mat

## DF

In [ ]:
df_main
# df.to_csv("df.csv", index = False)

# New Section

## Line Chart

In [ ]:
col_means = df_main.median(axis = 0)

In [ ]:
col_means

In [ ]:
# plt.plot(col_means)
# plt.ion()
# plt.show()
# plt.axis('auto')
# plt.gca().autoscale(enable=True)
# plt.gca().set(xscale='linear', yscale='linear')
# plt.gca().format_coord = lambda x, y: f'x={x:.2f}, y={y:.2f}'

# # keep plot window open
# while True:
#     plt.pause(0.1)

In [ ]:
# import plotly.graph_objs as go

In [ ]:
# fig = go.Figure()

In [ ]:
# fig.add_trace(go.Scatter(x = df['']))

In [ ]:
# print(type(df))

## Cloud percentage

In [ ]:
cloudFilter = merged_gdf[merged_gdf.CLOUDY_PIXEL_PERCENTAGE>50].timestamp.unique(), merged_gdf[merged_gdf.ndvi.isna()].timestamp.unique()
cloudFilter

In [ ]:
merged_gdf['Month']  = merged_gdf.timestamp.dt.month_name()
merged_gdf['Month_Numb']  = merged_gdf.timestamp.dt.month

In [ ]:
monthdf = merged_gdf[merged_gdf.CLOUDY_PIXEL_PERCENTAGE>30].Month_Numb.value_counts()/merged_gdf.Month_Numb.value_counts() * 100
monthdf.fillna(0)

In [ ]:
# gdf_1st520 = ee.FeatureCollection(merged_data.toList(520))

In [ ]:
# from google.colab import drive
# drive.mount('drive')

In [ ]:
# start_export = datetime.datetime.now()
# merged_gdf.to_csv('datagetvalue_merged_gdf.csv')
# !cp datagetvalue_merged_gdf.csv "/content/drive/MyDrive/iFarmer_data"
# datetime.datetime.now() - start_export

# Merged Data

In [ ]:
# if i == 0:
#   merged_data = ptsTopoStats
# else:
#   merged_data = merged_data.merge(ptsTopoStats)

In [ ]:
# task = ee.batch.Export.table.toDrive(collection = merged_data,  # an ee.Image object.
#                                      description = 'shorishabari_Demo_colab_Feb2128_all',
#                                      folder='iFarmer_data',
#                                      selectors = ['ndvi', 'ndre', 'msavi', 'ndmi', 'timestamp', '.geo'])

In [ ]:
# task.start()

In [ ]:
# task.status()

In [ ]:
# roi_test = [[89.47687222,25.79155833],
#                   [89.47677222,25.79167222],
#                   [89.47648333,25.79143889],
#                   [89.47658333,25.79134444],
#                   [89.47687222,25.79155833]]
# # ee.Geometry.Rectangle([-80.05, 25.79, -79.76, 26.07])

# # Define the time range of interest
# start_date = '2021-01-01'
# end_date = '2021-04-30'

# # Load the Sentinel-1 dataset
# sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')

# # Filter the data to include only the desired polarization and date range
# vh_imgs = sentinel1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
#                  .filterDate(start_date, end_date) \
#                  .filterBounds(roi_test)

# # Select the first image from the filtered collection
# vh_img = ee.Image(vh_imgs.first())

# # Get the radar backscatter data for the VH polarization
# vh_backscatter = vh_img.select('VH')

# # Get the data as a numpy array
# vh_backscatter_array = vh_backscatter.getDownloadUrl({
#     'scale': 10,
#     'crs': 'EPSG:4326',
#     'region': roi_test
# })

In [ ]:
# roi_test.getInfo()['coordinates'][0]

# Farm Scoring

In [ ]:
col_means = df_main.median(axis = 0)
col_means = col_means.astype(str)

In [ ]:
col_means

In [ ]:
col_means_df = col_means.to_frame('NDRE').reset_index().rename(columns={'index': 'Date'})

print(col_means_df)

In [ ]:
col_means_df['Date'] = pd.to_datetime(col_means_df['Date'])
col_means_df['NDRE'] = col_means_df['NDRE'].astype(float)
col_means_df['DAT'] = col_means_df['Date'].apply(lambda x: (x - reference_date).days)
col_means_df

In [ ]:
# Define the bin edges
bins = [40, 65, 95, 115]
# Bin the values in the DAT column
col_means_df['DAT_bin'] = pd.cut(col_means_df['DAT'], bins=bins)
# Group the data by the DAT_bin column and calculate the mean of the Value column for each group
avg_df = col_means_df.groupby('DAT_bin')['NDRE'].median().reset_index()
avg_df

In [ ]:
avg_df['DAT_bin'] = avg_df['DAT_bin'].astype(str)

## Conditons

In [ ]:
def condition_1(x):
    if x >= 0.4:
        return 10
    elif x >= 0.35:
        return 9
    elif x >= 0.3:
        return 8
    elif x >= 0.25:
        return 7
    elif x >= 0.2:
        return 6
    elif x >= 0.15:
        return 5
    else:
        return 3

def condition_2(x):
    if x >= 0.5:
        return 10
    elif x >= 0.45:
        return 9
    elif x >= 0.4:
        return 8
    elif x >= 0.35:
        return 7
    elif x >= 0.3:
        return 6
    elif x >= 0.25:
        return 5
    else:
        return 3

def condition_3(x):
    if x >= 0.45:
        return 10
    elif x >= 0.4:
        return 9
    elif x >= 0.35:
        return 8
    elif x >= 0.3:
        return 7
    elif x >= 0.25:
        return 6
    elif x >= 0.2:
        return 5
    else:
        return 3

In [ ]:
# Define a function that applies different conditions based on the value in the DAT_bin column
def apply_conditions(df):
    conditions = [
        (df['DAT_bin'] == '(40, 65]'),
        (df['DAT_bin'] == '(65, 95]'),
        (df['DAT_bin'] == '(95, 115]')
    ]

    choices = [df['NDRE'].apply(condition_1), df['NDRE'].apply(condition_2), df['NDRE'].apply(condition_3)]

    df['Score'] = np.select(conditions, choices)

# Call the function with the avg_df DataFrame as an argument
apply_conditions(avg_df)

print(avg_df)

## Farm Scoring

In [ ]:
# Define the weights
weights = [0.2, 0.65, 0.15]

# Calculate the weighted average of the values in the Status column
farm_score_df = (avg_df['Score'] * weights).sum()

print('Farm Score DF = ',farm_score_df)

In [ ]:
farm_score = (farm_score_mat*.4 + farm_score_df*.6)
print('Farm Score = ', farm_score)